## 7.12 分支和 switch 语句

现代微处理器的高速度是通过流水线获得的。流水线中，指令在执行前分成几个阶段来获取和解码。
然而，流水线结构有一个大问题。一旦代码中有分支出现（例如，`if-else`），微处理器并不能预先知道该把两个分支中的哪一个送入流水线。
如果错误的分支被喂给了流水线，该错误直到10至20个时钟周期后才能被发现，这期间所有的取指，解码，甚至推测性指令执行工作都被浪费掉了。
结果是，只要微处理器将分支喂给流水线并稍后发现它选错了分支，微处理器就会浪费小部分时钟周期。

微处理器设计者已经竭尽全力减轻了该问题的后果，使用的最重要的方法是分支预测。
现代微处理器使用先进的算法，根据该分支及附近分支的历史数据来预测分支的走向。
不同的微处理器使用的分支预测算法并不相同。在手册3：“Inter, AMD 和 VIA CPU 的微处理器架构”中有对这些算法的描述。

在微处理器做出正确预测的情况下，分支指令通常需要0-2个时钟周期。从分支错误预测中恢复，所需的时间大约为12-25个时钟周期，具体取决于处理器。 这被称为分支错误预测惩罚。

在大多数情况下的正确预测分支是相对代价较小，但如果它们经常被错误预测则代价高昂。
- 显然，某个分支总重复相同的走法，预测会很准。
- 如果某个大多数时间走法相同，但很少的情况下走法不同，该分支只会在不同走法时错误预测。
- 如果一个分支多次重复一种方式，然后多次重复另一种，只有在变化时才会被错误预测。
- 如果分支遵循简单的周期性的模式，也可以被很好的预测，当它是不包含其它分支，或者只包含很少其它分支的内层循环时。简单周期性模式的一个例子是，一种方式走两次，另一种方式走一次。然后，第一种方式走两次，第二种方式走三次，等等。
- 最糟糕的情况是一个分支随机选择某种一条路，或者以50%-50%的概率走任意一条。 这样的分支将在50%的时间内被错误预测。

for循环或while循环也是一种分支。每次循环迭代后，会决定是重复循环还是退出。
如果循环计数很小并且不变，循环分支通常可以很好的被预测。
可以完美预测的最大循环计数通常介于9到64之间，具体取决于处理器。
仅仅某些处理器上可以较好地预测嵌套循环。
在许多处理器上，一个循环内包含多个分支，不能很好地被预测。

switch 语句是一种超过两条路的分支。
下面情形 switch 语句最为高效：
- 每个 case 的标签遵循一个序列
- 在该序列中，某个标签的当前值等于上一个标签加一。

这是因为，这种情况可以使用跳转目标表来实现。
具有许多标签的 switch 语句，并且标签的值离得很远，该情形是很低效的，因为编译器必须将其转换为分支树。

在较老的处理器上，对于带有顺序标签的switch语句，CPU仅仅重复上次执行时的方式来进行分支预测。
因此，只要当程序实际走入分支中的另一条路时，都肯定会被错误预测。

较新的处理器有时能够较好的预测switch语句，
- 如果它遵循简单的周期性模式，
- 或者它与前面的分支相关，并且不同目标的数量很小。(原文：if it is correlated with preceding branches and the number of different targets is small.)

**在程序的关键部分，分支和switch语句的数量最好保持较小，特别是如果分支的可预测性很差。**
如果可以消除分支，则展开循环可能很有用。下一段将会细讲。

分支和函数调用的目标（地址）保存在称为分支目标缓冲区的特殊高速缓存中。
如果程序具有许多分支或函数调用，则可能在分支目标缓冲区中发生争用。
这种争用的后果是，即使在其它情况下，分支预测得很好，分支也可能被错误预测。
甚至是函数调用也可能因此被错误预测。
**因此，在代码的关键部分，具有许多很多分支和函数调用的程序可能导致错误的分支预测。**

在某些情况下，可以通过查找表来替换难以预测的分支。 例如：

```cpp
// Example 7.29a
float a; bool b;
a = b ? 1.5f : 2.6f;
```

这里`?:`操作符就是一个分支。如果它很难被预测，就用一个查找表来代替：

```cpp
// Example 7.29b
float a; bool b = 0;
const float lookup[2] = {2.6f, 1.5f};
a = lookup[b];
```
如果将bool用作数组索引，则确保它正确的初始化，或自可靠来源非常重要，这样它就不能具有0或1以外的其他值。请参见第34页。

在某些情况下，编译器可以通过“条件移动”指令自动替换分支，具体取决于指定的指令集。

第138页和第139页的示例显示了减少分支数量的各种方法。

手册3：“Intel，AMD和VIA CPU的微体系结构”提供了有关不同微处理器中分支预测的更多详细信息。